In [1]:
import pynq
from dishsoap import StreamController, TestDmaStreamOverlay
streamer = TestDmaStreamOverlay()

In [2]:
results = pynq.allocate(shape=(125,), dtype='u8')
streamer.fill_buffer(5, results)
print(results)

[  5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22
  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40
  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58
  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76
  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94
  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112
 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129]


## Benchmark

In [26]:
from time import time

def do_benchmark(size = 16, iters = 1) -> dict:
    t0 = time()
    bench_array = pynq.allocate(shape=(size,),dtype='u8')
    t1 = time()
    for _ in range(iters):
        streamer.fill_buffer(1, bench_array)
    t2 = time()
    return {'alloc': t1-t0, 'fill': t2-t1, 'total':t2-t0}

#size = 2**8
#iters = 2
#times = do_benchmark(size)
#for category in ('alloc', 'fill', 'total'):
#    print(f'Runtime ({category}): {times[category]:4.6f} s')

results = []

for size_exp in range(13):
    for iters_exp in range(7):
        size = 2**size_exp
        iters = 10**iters_exp
        times = do_benchmark(size, iters)
        times['size'] = size
        times['iters'] = iters
        results.append(times)
        print(f"Runtime (): {times['fill']:4.6f} s")

import pickle
with open('results.pickle', 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)
    print(f"Saved results to 'results.pickle'")

Runtime (): 0.000768 s
Runtime (): 0.005790 s
Runtime (): 0.048535 s
Runtime (): 0.482596 s
Runtime (): 4.726547 s
Runtime (): 47.486923 s
Runtime (): 471.902651 s
Runtime (): 0.000752 s
Runtime (): 0.006835 s
Runtime (): 0.047400 s
Runtime (): 0.480858 s
Runtime (): 4.743493 s
Runtime (): 46.971554 s
Runtime (): 469.093750 s
Runtime (): 0.000748 s
Runtime (): 0.005308 s
Runtime (): 0.050078 s
Runtime (): 0.475248 s
Runtime (): 4.695933 s
Runtime (): 46.717793 s
Runtime (): 468.219835 s
Runtime (): 0.000727 s
Runtime (): 0.005247 s
Runtime (): 0.047610 s
Runtime (): 0.477453 s
Runtime (): 4.693148 s
Runtime (): 46.599677 s
Runtime (): 468.253398 s
Runtime (): 0.000761 s
Runtime (): 0.005519 s
Runtime (): 0.048013 s
Runtime (): 0.485601 s
Runtime (): 4.735056 s
Runtime (): 47.266232 s
Runtime (): 471.172089 s
Runtime (): 0.000747 s
Runtime (): 0.005680 s
Runtime (): 0.049703 s
Runtime (): 0.480694 s
Runtime (): 4.726023 s
Runtime (): 46.680385 s
Runtime (): 466.471002 s
Runtime (): 0.00

In [30]:
big_test = pynq.allocate(shape=(5000,), dtype='u8')
streamer.fill_buffer(60000,big_test)
print(big_test)

[60000 60001 60002 ... 64997 64998 64999]


## Results:

Speeds not including time to allocate the buffer on PS-side

In [48]:
for res in results:
    bits = res['iters'] * res['size'] * 64
    mbps = bits / ((10**6) * res['fill'])
    MBs  = bits / ((2**20) * 8 * res['fill'])
    info = ''
    info += f"Iters: {res['iters']:<10,d} "
    info += f"Size: {res['size']*64:8,d} "
    info += f"bytes  Speed: {mbps:9.3f} Mbps "
    info += f" or {MBs:7.3f} MB/s"
    print(info)

Iters: 1          Size:       64 bytes  Speed:     0.083 Mbps  or   0.010 MB/s
Iters: 10         Size:       64 bytes  Speed:     0.111 Mbps  or   0.013 MB/s
Iters: 100        Size:       64 bytes  Speed:     0.132 Mbps  or   0.016 MB/s
Iters: 1,000      Size:       64 bytes  Speed:     0.133 Mbps  or   0.016 MB/s
Iters: 10,000     Size:       64 bytes  Speed:     0.135 Mbps  or   0.016 MB/s
Iters: 100,000    Size:       64 bytes  Speed:     0.135 Mbps  or   0.016 MB/s
Iters: 1,000,000  Size:       64 bytes  Speed:     0.136 Mbps  or   0.016 MB/s
Iters: 1          Size:      128 bytes  Speed:     0.170 Mbps  or   0.020 MB/s
Iters: 10         Size:      128 bytes  Speed:     0.187 Mbps  or   0.022 MB/s
Iters: 100        Size:      128 bytes  Speed:     0.270 Mbps  or   0.032 MB/s
Iters: 1,000      Size:      128 bytes  Speed:     0.266 Mbps  or   0.032 MB/s
Iters: 10,000     Size:      128 bytes  Speed:     0.270 Mbps  or   0.032 MB/s
Iters: 100,000    Size:      128 bytes  Speed:     0